# 식신 - 양재역으로 검색
- 형식, 주요메뉴 등
- 한번더 들어가야된다 메뉴 안속으로

In [41]:
import requests
import pandas as pd
from urllib.parse import quote #한글데이터 주소에 쓰려면
from bs4 import BeautifulSoup

In [42]:
header = {'User-Agent' :    
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

## 데이터 가져오기

In [43]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords={quote("양재역")}'
req = requests.get(url)
html = req.text
#header가 필요없다 그냥 허락됨

- 찾으려고 하는 데이터 태그 찾기

In [44]:
soup = BeautifulSoup(html, 'html.parser')
lis = soup.select_one('div.listTy1').find_all('li') #listTy1이 또있어서 숫자 많다
len(lis)

60

In [45]:
###schMove1 > div.listTy1 > ul > li:nth-child(1)
ul = soup.select('div.listTy1 > ul > li') #클래스 앞에는 . 붙인다
len(ul)

12

In [46]:
li =ul[0]
href = li.select_one('a')['href']
href
##schMove1 > div.listTy1 > ul > li:nth-child(2) > div > a

'/P/358859'

In [64]:
name = soup.select_one('.box_tit strong').string
name

'그릭슈바인 양재역점'

In [47]:
soup.select('.store')[0].string

'그릭슈바인 양재역점'

In [48]:
sub_url = base_url + href #메인페이 가서 해당페이지로 이동한다
req = requests.get(sub_url)
sub_soup = BeautifulSoup(req.text, 'html.parser')
# 이름, 업종, 전화번호, 주소

In [49]:
store = sub_soup.select_one('.title')
store

<div class="title" data-reactid="54"><div class="store_name_score" data-reactid="55"><h3 data-reactid="56"><!-- react-text: 57 -->그릭슈바인 양재역점<!-- /react-text --><strong data-reactid="58">3.6</strong><div class="ico_both02" data-reactid="59"><span class="ico_grade02" data-reactid="60"><em data-reactid="61">HOT</em></span></div></h3><p data-reactid="62">독일식 맥주 펍</p><ul data-reactid="63"><li data-reactid="64"><span data-reactid="65">12</span></li><li data-reactid="66"><span data-reactid="67">16</span></li><li data-reactid="68"><span data-reactid="69">13333</span></li><li data-reactid="70"><span data-reactid="71">25</span></li><li class="btn_dining_use" data-reactid="72"></li></ul></div></div>

In [50]:
store = sub_soup.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [51]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [52]:
info.find(score)

10

In [53]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [54]:
##contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > p
menu = sub_soup.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [55]:
tel =sub_soup.select_one('.p_tel > p').get_text()
tel

'02-572-6100'

In [56]:
addr =sub_soup.select_one('.txt_adr').string
addr

'서울특별시 서초구 강남대로 224'

In [72]:
name_list, score_list, menu_list, tel_list, addr_list =[], [], [], [], []
for i, li in enumerate(ul):
    href = li.select_one('a')['href']
    sub_url = base_url + href #메인페이 가서 해당페이지로 이동한다
    req = requests.get(sub_url)
    sub_soup = BeautifulSoup(req.text, 'html.parser')
    
  #  store = sub_soup.select_one('.title')
  #  info = store.find('h3').get_text() 
  #  score = str(store.select_one('h3 > strong').string) 
    
  #  name = info[:info.find(score)]
    menu = sub_soup.select('.store_info p')[1].get_text()
    tel =sub_soup.select_one('.p_tel > p').get_text()
    addr =sub_soup.select_one('.txt_adr').string
    
    score = li.select_one('em').string
    name = soup.select('.store')[i].string

    name_list.append(name)
    score_list.append(score)
    menu_list.append(menu)
    tel_list.append(tel)
    addr_list.append(addr)

In [73]:
df = pd.DataFrame({
    '이름' : name_list,
    '점수' : score_list,
    '메뉴' : menu_list,
    '전화' : tel_list,
    '주소' : addr_list
})
df

,이름,점수,메뉴,전화,주소
0,그릭슈바인 양재역점,3.6,나이트라이프 > 맥주/호프,02-572-6100,서울특별시 서초구 강남대로 224
1,미스터브리즈 양재역점,3.4,세계음식 > 카페/커피숍,02-576-4333,서울 서초구 강남대로34길 6
2,동신참치 양재역점,3.0,한국음식 > 회,02-529-7130,서울 서초구 강남대로 213
3,새우닭 양재역점,평가중,세계음식 > 씨푸드,02-575-6088,서울특별시 서초구 남부순환로356길 52 1층
4,양재역 옛날짜장,평가중,세계음식 > 중국음식,031-503-2550,경기도 시흥시 공단1대로 13
5,영동족발 3호점,4.1,한국음식 > 족발/보쌈,02-575-0250,서울특별시 서초구 남부순환로 2628-18
6,황재벌,3.9,한국음식 > 장어구이/꼼장어,02-3471-5454,서울특별시 서초구 남부순환로347길 42-4 신화빌딩
7,김영모과자점 도곡타워점,4.1,세계음식 > 베이커리/제과점,02-3460-2005,서울특별시 강남구 언주로30길 10 현대비젼21
8,소호정 본점,4.0,한국음식 > 라면/칼국수/국수/수제비,02-579-7282,서울특별시 서초구 논현로 27
9,산동칼국수,3.9,한국음식 > 라면/칼국수/국수/수제비,02-3473-7972,서울특별시 서초구 강남대로37길 63
